## Frequentist Estimation of Limit/Interval

The following example show how to compute a frequentist limit. 

The limit is computed with a techinique called hypothesis test inversion.
An hypothesis test is performed for fixed values of the parameter of interest (e.g. $\mu$). 

The test is built with profile log-likelihood ratio as test statistics and using as $H_0$, the null hypothesis, the $S+B$ model vs $H_1$, the B$ only model. 

For each $\mu$ value, a p-value, which is the significance $\alpha$ of the test, is obtained and plotted as function of $\mu$. 

The interval at a given confidence level, CL = $1-\alpha_0$, is then obtained by looking at the values of $\mu$ which have a p-value equal to $\alpha_0$.

Different type of interval can be computed depending on the test statistics used. 

-  For upper limit the used test statistics is the one-side profiled likelihood :

$$q_\mu = - 2 \log \frac { L( x \ | \ \mu , \hat{\hat{\nu} } ) }{  L( x \ | \ \hat{\mu} , \hat{\nu}  ) } \  \  \ \mathrm{for} \  \  
\hat{\mu} \le \mu_{TEST}$$
$$q_\mu = 0     \hspace{3cm}     \ \mathrm{for} \  \  
\hat{\mu} \gt \mu_{TEST}$$

   The limit is then computed using the $CL_s$ procedure, i.e. the actual p-value used to build the scan curve is 
   $$CL_s = \frac{ CL_{S+B} } {CL_B } $$
   
- For Feldman-Cousins interval the two-sided profile likelihood is used:    

$$q_\mu = - 2 \log \frac { L( x \ | \ \mu , \hat{\hat{\nu} } ) }{  L( x \ | \ \hat{\mu} , \hat{\nu}  ) }$$

   for every obtained  $ \hat{\mu}$ value.
   
The code can also compute expected limits using the Asimov data set. 

At the beginning we compute the limit using the asympotic approximation of the distribution of the profile likelihood test statistics and we don't need to generate pseudo-experiments.  See paper *G. Cowan et al., arXiv:1007.1727,EPJC 71 (2011)* 

The asymptotic approximation is used only for computing $CL_s$ limits

Later we compute the full frequentist limit using pseudo-experiments and profiling nuisance parameter if are present. 


In [ ]:
using namespace RooStats;

In [ ]:
HypoTestInverter * calc = nullptr;
HypoTestInverterResult * result = nullptr; 
HypoTestInverterPlot * plot = nullptr; 
ProfileLikelihoodTestStat * testStat = nullptr; 
ToyMCSampler * toymcs = nullptr; 
SamplingDistPlot * distplot = nullptr;
TCanvas * c1 = nullptr; 
TCanvas * c2 = nullptr; 

#####  Reading the model (Workspace) from input file

First part is just to access the workspace file and retrieve the model and the data 

In [ ]:
TString fileName = "CountingModelFixedB.root";
TString workspaceName = "w";
TString modelConfigName = "ModelConfig";
TString dataName = "data";
TString integrationType = "";  
// Use Minuit2 for minimization
ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");
// open the file 
auto file = TFile::Open(fileName);
// read workspace from the file 
auto w =  (RooWorkspace*) file->Get(workspaceName);
w->Print();
auto sbModel = (RooStats::ModelConfig*) w->obj(modelConfigName);
sbModel->Print();
auto  data = w->data(dataName);
auto poi = (RooRealVar*) sbModel->GetParametersOfInterest()->first();

Print information about the number of events and background (works only in case of the specific counting model)

In [ ]:
std::cout << "number of observed events in the data : " << std::endl;
auto args = data->get(0);
if (args) args->Print("V");
else  Error("BayesianNumerical","Input data does not contain any event");
std::cout << "number of background events " << w->var("b")->getVal() << std::endl;

##### Make the b Model by cloning the b model and use a value = 0 for the parameter of interest

In [ ]:
sbModel->SetName("S+B Model");
auto bModel = (RooStats::ModelConfig*) sbModel->Clone("B Model");
poi->setVal(0);
bModel->SetSnapshot( *poi  );
sbModel->GetSnapshot()->Print("V");

###  Computation using Asymptotic formula

#### we create first the hypothesis test calculator: the `AsymptoticCalculator`
 
Note that in this case the NULL model is the S+B model. 

In [ ]:
// RooStats::AsymptoticCalculator::SetPrintLevel(-1);  // to switch off print level 
RooStats::AsymptoticCalculator  asymCalc(*data, *bModel, *sbModel);

In [ ]:
asymCalc.SetOneSided(true);  // for one-side test (compute a limit)
//asymCalc.SetPrintLevel(-1);  // to suppress print level 

####  Create the HypoTestInverter calculator and configure it 

In [ ]:
calc = new RooStats::HypoTestInverter(asymCalc); 

Configure the calculator

In [ ]:
calc->SetConfidenceLevel(0.90);
calc->UseCLs(true); 
calc->SetVerbose(false);

calc->SetFixedScan(10,0.0,6.0); // set number of points , xmin and xmax 

asymCalc.SetOneSided(true);  // for one-side test (compute a limit)
asymCalc.SetPrintLevel(-1); 

#### Run now the calculator to compute the interval 

In [ ]:
result = calc->GetInterval();

In [ ]:
cout << 100*calc->ConfidenceLevel() << "% interval is : ["<< result->LowerLimit() << ", "<< result->UpperLimit() <<"] "<<endl;

std::cout << "The computed upper limit is: " << result->UpperLimit() << std::endl;
  
  // compute expected limit
  std::cout << "Expected upper limits, using the B (alternate) model : " << std::endl;
  std::cout << " expected limit (median) " << result->GetExpectedUpperLimit(0) << std::endl;
  std::cout << " expected limit (-1 sig) " << result->GetExpectedUpperLimit(-1) << std::endl;
  std::cout << " expected limit (+1 sig) " << result->GetExpectedUpperLimit(1) << std::endl;
  std::cout << " expected limit (-2 sig) " << result->GetExpectedUpperLimit(-2) << std::endl;
  std::cout << " expected limit (+2 sig) " << result->GetExpectedUpperLimit(2) << std::endl;

#### Plot now the result of the scan of the hypothesis tests

In [ ]:
plot = new RooStats::HypoTestInverterPlot("HTI_Result_Plot","HypoTest Scan Result",result);
plot->Draw("CLb 2CL");  // plot also CLb and CLs+b 
gPad->Draw();

### Frequentist Calculator

We run now on the same model the FrequentistCalculator. The Frequentist Calculator uses the test statistic distributions obtained with pseudo-experiments.

#### Set the type of limit: CLs or FC ? 

In [ ]:
bool useCLs = true; 

In [ ]:
RooStats::FrequentistCalculator   fc(*data, *bModel, *sbModel);

We need also to specify the test statistics type. Here are some possible test statistics to use 

In [ ]:
testStat = new RooStats::ProfileLikelihoodTestStat(*sbModel->GetPdf());

In [ ]:
toymcs = (RooStats::ToyMCSampler*)fc.GetTestStatSampler();
toymcs->SetTestStatistic(testStat);

In [ ]:
// for number counting experiments (i.e. when we have only one event per toy)
// in general shape cases are extended model
if (!sbModel->GetPdf()->canBeExtended())
    toymcs->SetNEventsPerToy(1);

We create and configure the RooStats calculators to compute the frequentist interval using the inversion of the hypothesis test

In [ ]:
calc = new RooStats::HypoTestInverter(fc);

#### Configure the Calculator

We can configure the calculator to compute CLs limits or Feldman-Cousins intervals.
These first lines are for CLs limits.

In [ ]:
// for CLS
if (useCLs) {
    calc->UseCLs(true);
    testStat->SetOneSided(true);
}

These are for Feldman-Cousins intervals. Uncomment the lines if you want to compute them 

In [ ]:
// uncomment these lines for Feldman Cousins intervals
if (!useCLs) {
  calc->Clear();
  calc->UseCLs(false);
  testStat->SetOneSided(false);
}

We configure the Frequentist calculator by specifying the number of toys for the two hypothesis 

In [ ]:
fc.SetToys(1000,1000);    // 1000 for null (S+B) and 500 for alt (B) 

#### Run the toys to compute the test statistic distributions

Run now the calculator. We configure also the CL level and the range and number of points to scan. 
It can take some time... be patient 

In [ ]:
calc->Clear(); // needed if running a second time
calc->SetConfidenceLevel(0.90);  
calc->SetFixedScan(10,0.1,5.1); // set number of points , xmin and xmax 
result = calc->GetInterval();

#### Examine the result

We examine now the result of the calculator. 
From the scan we can also compute in addition to the observed interval limits, also the expected values (median and +/- 1,2 sigma), which are computed using the same pseudo-experiments used to compute the observed limits. 

In [ ]:
cout << 100*calc->ConfidenceLevel() << "% interval is : ["<< result->LowerLimit() << ", "<< result->UpperLimit() <<"] "<<endl;

std::cout << "The computed upper limit is: " << result->UpperLimit() << std::endl;
  
  // compute expected limit
  std::cout << "Expected upper limits, using the B (alternate) model : " << std::endl;
  std::cout << " expected limit (median) " << result->GetExpectedUpperLimit(0) << std::endl;
  std::cout << " expected limit (-1 sig) " << result->GetExpectedUpperLimit(-1) << std::endl;
  std::cout << " expected limit (+1 sig) " << result->GetExpectedUpperLimit(1) << std::endl;
  std::cout << " expected limit (-2 sig) " << result->GetExpectedUpperLimit(-2) << std::endl;
  std::cout << " expected limit (+2 sig) " << result->GetExpectedUpperLimit(2) << std::endl;

#### Plot Result of Scan

In [ ]:
c1 = new TCanvas("c1","HypoTest Scan Result");
plot = new RooStats::HypoTestInverterPlot("HTI_Result_Plot","HypoTest Scan Result",result);
if (useCLs) 
 plot->Draw("CLb 2CL");  // plot also CLb and CLs+b 
else 
 plot->Draw("OBS");  // draw only the observed value   
     
c1->SetLogy(0);
c1->Draw();

#### Plot Result of Test Statistic Distribution

Plot now the test statistics distributions for some of the scanned points

In [ ]:
//%jsroot on

In [ ]:
c2 = new TCanvas("c2","Test Statistic Distributions",1000,800);
c2->Divide(2,2);
for (int i = 1; i/2 < 5 && i < result->ArraySize(); i+=2){
    c2->cd(i/2+1);
    distplot = plot->MakeTestStatPlot(i);
    distplot->SetLogYaxis(true);
    distplot->Draw();
    text = new TText(.6,.6,TString::Format("%s = %f",poi->GetName(),result->GetXValue(i)));
    text->SetNDC();
    text->Draw();    
}
c2->Draw();